# Amazon API Data Extraction Test (Corrected)

This notebook tests the corrected, two-step approach for Amazon product discovery:
1.  **`SearchItems`**: To get a list of candidate ASINs.
2.  **`GetItems`**: To enrich those ASINs with detailed data.

This demonstrates the optimal way to fetch product data and reveals that `CustomerReviews` are not available for this API account.


In [1]:
# Cell 1: Install Dependencies & Imports

import sys
!{sys.executable} -m pip install python-dotenv python-amazon-paapi gspread google-auth-oauthlib

import json
import time
from dotenv import load_dotenv

# Load credentials from .env file
load_dotenv()

# Import the specific Amazon PA API components we need
from amazon_paapi.sdk.api.default_api import DefaultApi
from amazon_paapi.sdk.models.partner_type import PartnerType
from amazon_paapi.sdk.models.search_items_request import SearchItemsRequest
from amazon_paapi.sdk.models.search_items_resource import SearchItemsResource
from amazon_paapi.sdk.models.get_items_request import GetItemsRequest
from amazon_paapi.sdk.models.get_items_resource import GetItemsResource
from amazon_paapi.sdk.models.delivery_flag import DeliveryFlag
from amazon_paapi.sdk.rest import ApiException

print("✅ Dependencies and imports are ready.")


✅ Dependencies and imports are ready.


In [ ]:
# Cell 2: Configuration and API Client Setup

from config import conf

# --- PAAPI Credentials & Configuration ---
ACCESS_KEY = conf.amazon.access_key
SECRET_KEY = conf.amazon.secret_key
PARTNER_TAG = conf.amazon.associate_tag
HOST = "webservices.amazon.it"
REGION = "eu-west-1"

# --- API Functions ---

def search_api(browse_node_id, item_page=1, min_price=None, max_price=None, 
               delivery_flags=None, min_reviews_rating=None, min_saving_percent=None):
    """
    Calls the SearchItems API for a given category and page with optional filters.
    
    Args:
        browse_node_id: Browse node ID for the category
        item_page: Page number (default: 1)
        min_price: Minimum price in euros (float, e.g., 10.50)
        max_price: Maximum price in euros (float, e.g., 100.00)
        delivery_flags: List of delivery flags (e.g., ["FulfilledByAmazon"])
        min_reviews_rating: Minimum star rating (int, 1-5, e.g., 4)
        min_saving_percent: Minimum discount percentage (int, e.g., 10 for 10%)
    """
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    resources = [
        SearchItemsResource.ITEMINFO_TITLE,
        SearchItemsResource.OFFERS_SUMMARIES_LOWESTPRICE,
        SearchItemsResource.IMAGES_PRIMARY_LARGE,
        SearchItemsResource.CUSTOMERREVIEWS_COUNT,
        SearchItemsResource.CUSTOMERREVIEWS_STARRATING,
        SearchItemsResource.BROWSENODEINFO_WEBSITESALESRANK,
        SearchItemsResource.BROWSENODEINFO_BROWSENODES,
    ]
    
    # Convert price to cents for API
    min_price_cents = int(min_price * 100) if min_price else None
    max_price_cents = int(max_price * 100) if max_price else None
    
    # Handle delivery flags
    delivery_flag_list = []
    if delivery_flags:
        if "FulfilledByAmazon" in delivery_flags:
            delivery_flag_list.append(DeliveryFlag.FULFILLEDBYAMAZON)
    
    try:
        request = SearchItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            browse_node_id=browse_node_id,
            item_page=item_page,
            item_count=10,
            sort_by="Featured",
            resources=resources,
            min_price=min_price_cents,
            max_price=max_price_cents,
            min_reviews_rating=min_reviews_rating,
            min_saving_percent=min_saving_percent,
            delivery_flags=delivery_flag_list if delivery_flag_list else None
        )
        response = api_client.search_items(request)
        return response.to_dict()
    except ApiException as e:
        print(f"API Error on page {item_page} for node {browse_node_id}: {e.reason}")
        return None

def get_items_details(asins):
    """Fetches detailed information for a list of ASINs using the GetItems API."""
    if not asins:
        return []
    
    print(f"\n--- Enriching {len(asins)} items with GetItems API ---")
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # Build resources list safely - only add resources that exist
    resources = []
    
    # Always available resources
    safe_resources = [
        'ITEMINFO_TITLE',
        'OFFERS_LISTINGS_PRICE',
        'IMAGES_PRIMARY_LARGE',
        'CUSTOMERREVIEWS_COUNT',
        'CUSTOMERREVIEWS_STARRATING',
        'ITEMINFO_FEATURES',
        'BROWSENODEINFO_WEBSITESALESRANK'
    ]
    
    # Optional resources - try to add them if they exist
    optional_resources = [
        'ITEMINFO_PRODUCT_INFO',
        'ITEMINFO_TECHNICAL_INFO',
        'ITEMINFO_CONTENT_INFO',
        'ITEMINFO_CLASSIFICATIONS',
        'ITEMINFO_BYLINE_INFO',
        'ITEMINFO_EXTERNAL_IDS',
        'ITEMINFO_MANUFACTURE_INFO',
        'ITEMINFO_TRADE_IN_INFO'
    ]
    
    # Add safe resources
    for resource_name in safe_resources:
        try:
            resource = getattr(GetItemsResource, resource_name)
            resources.append(resource)
        except AttributeError:
            print(f"Warning: Resource {resource_name} not available")
    
    # Try to add optional resources
    for resource_name in optional_resources:
        try:
            resource = getattr(GetItemsResource, resource_name)
            resources.append(resource)
        except AttributeError:
            pass  # Silently skip if not available

    asin_chunks = [asins[i:i + 10] for i in range(0, len(asins), 10)]
    enriched_items = []

    for i, chunk in enumerate(asin_chunks):
        print(f"Fetching details for batch {i+1}/{len(asin_chunks)}...")
        request = GetItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            item_ids=chunk,
            resources=resources
        )
        try:
            response = api_client.get_items(request).to_dict()
            if response and response.get("items_result") and response["items_result"].get("items"):
                enriched_items.extend(response["items_result"]["items"])
        except ApiException as e:
            print(f"API Error (GetItems) on batch {i+1}: {e.reason}")
        
        time.sleep(1)

    return enriched_items

def print_search_results(search_response):
    """Pretty print the search results from SearchItems API."""
    if not search_response:
        print("❌ No response received from API.")
        return
    
    if "search_result" not in search_response:
        print("❌ No search_result in response.")
        print(f"Response keys: {list(search_response.keys())}")
        return
    
    search_result = search_response["search_result"]
    
    total_results = search_result.get("total_result_count", "Unknown")
    search_index = search_result.get("search_index", "Unknown")
    browse_node_id = search_result.get("browse_node", {}).get("id", "Unknown") if search_result.get("browse_node") else "Unknown"
    browse_node_name = search_result.get("browse_node", {}).get("display_name", "Unknown") if search_result.get("browse_node") else "Unknown"
    
    print(f"\n{'='*80}")
    print(f"📊 Search Results Summary")
    print(f"{'='*80}")
    print(f"Total Results: {total_results}")
    print(f"Search Index: {search_index}")
    print(f"Browse Node ID: {browse_node_id}")
    print(f"Browse Node Name: {browse_node_name}")
    print(f"{'='*80}\n")
    
    items = search_result.get("items", [])
    if not items:
        print("⚠️  No items found in search results.")
        return
    
    print(f"Found {len(items)} items on this page:\n")
    
    for idx, item in enumerate(items, 1):
        asin = item.get("asin", "N/A")
        title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")
        
        price = "N/A"
        savings = "N/A"
        if item.get("offers") and item["offers"].get("summaries"):
            summaries = item["offers"]["summaries"]
            if summaries and len(summaries) > 0:
                lowest_price = summaries[0].get("lowest_price", {})
                price = lowest_price.get("display_amount", "N/A")
                if "saving" in summaries[0] and summaries[0]["saving"]:
                    savings = summaries[0]["saving"].get("display_amount", "N/A")
        
        image_url = "N/A"
        if item.get("images") and item["images"].get("primary") and item["images"]["primary"].get("large"):
            image_url = item["images"]["primary"]["large"].get("url", "N/A")
        
        review_count = "N/A"
        star_rating = "N/A"
        if item.get("customer_reviews"):
            review_count = item["customer_reviews"].get("count", "N/A")
            if item["customer_reviews"].get("star_rating"):
                star_rating = item["customer_reviews"]["star_rating"].get("display_value", "N/A")
        
        sales_rank = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("website_sales_rank"):
            sales_rank_obj = item["browse_node_info"]["website_sales_rank"]
            if isinstance(sales_rank_obj, dict):
                sales_rank = sales_rank_obj.get("sales_rank", "N/A")
        
        browse_nodes = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("browse_nodes"):
            browse_nodes_list = item["browse_node_info"]["browse_nodes"]
            if browse_nodes_list:
                browse_nodes = browse_nodes_list[0].get("display_name", "N/A")
        
        detail_url = item.get("detail_page_url", "N/A")
        
        print(f"{'─'*80}")
        print(f"Item #{idx}")
        print(f"{'─'*80}")
        print(f"ASIN:         {asin}")
        print(f"Title:        {title}")
        print(f"Price:        {price}")
        if savings != "N/A":
            print(f"Savings:      {savings}")
        print(f"Reviews:      {review_count} ({star_rating} stars)")
        print(f"Sales Rank:   {sales_rank}")
        print(f"Browse Node:  {browse_nodes}")
        if len(image_url) > 80:
            print(f"Image URL:    {image_url[:77]}...")
        else:
            print(f"Image URL:    {image_url}")
        if len(detail_url) > 80:
            print(f"Detail URL:   {detail_url[:77]}...")
        else:
            print(f"Detail URL:   {detail_url}")
        print()
    
    print(f"{'='*80}\n")

def print_enriched_items(enriched_items):
    """Pretty print the enriched items from GetItems API."""
    if not enriched_items:
        print("❌ No enriched items to display.")
        return
    
    print(f"\n{'='*80}")
    print(f"📦 Enriched Items Details")
    print(f"{'='*80}")
    print(f"Total Items: {len(enriched_items)}\n")
    
    for idx, item in enumerate(enriched_items, 1):
        asin = item.get("asin", "N/A")
        title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")
        
        price = "N/A"
        savings = "N/A"
        if item.get("offers") and item["offers"].get("listings"):
            listings = item["offers"]["listings"]
            if listings:
                price_obj = listings[0].get("price", {})
                price = price_obj.get("display_amount", "N/A")
                if price_obj.get("savings"):
                    savings = price_obj["savings"].get("display_amount", "N/A")
        
        image_url = "N/A"
        if item.get("images") and item["images"].get("primary") and item["images"]["primary"].get("large"):
            image_url = item["images"]["primary"]["large"].get("url", "N/A")
        
        review_count = "N/A"
        star_rating = "N/A"
        if item.get("customer_reviews"):
            review_count = item["customer_reviews"].get("count", "N/A")
            if item["customer_reviews"].get("star_rating"):
                star_rating = item["customer_reviews"]["star_rating"].get("display_value", "N/A")
        
        sales_rank = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("website_sales_rank"):
            sales_rank_obj = item["browse_node_info"]["website_sales_rank"]
            if isinstance(sales_rank_obj, dict):
                sales_rank = sales_rank_obj.get("sales_rank", "N/A")
        
        detail_url = item.get("detail_page_url", "N/A")
        
        # Features (bullet points) - safely get
        features = []
        try:
            features = item.get("item_info", {}).get("features", {}).get("display_values", [])
        except (AttributeError, KeyError, TypeError):
            pass
        
        # Product Info - safely get
        product_info = {}
        try:
            product_info = item.get("item_info", {}).get("product_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        color = product_info.get("color", {}).get("display_value", None) if product_info and product_info.get("color") else None
        size = product_info.get("size", {}).get("display_value", None) if product_info and product_info.get("size") else None
        unit_count = product_info.get("unit_count", {}).get("display_value", None) if product_info and product_info.get("unit_count") else None
        
        # Classifications - safely get
        classifications = {}
        try:
            classifications = item.get("item_info", {}).get("classifications", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        product_group = classifications.get("product_group", {}).get("display_value", None) if classifications and classifications.get("product_group") else None
        binding = classifications.get("binding", {}).get("display_value", None) if classifications and classifications.get("binding") else None
        
        # Content Info - safely get
        content_info = {}
        try:
            content_info = item.get("item_info", {}).get("content_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        pages_count = content_info.get("pages_count", {}).get("display_value", None) if content_info and content_info.get("pages_count") else None
        publication_date = content_info.get("publication_date", {}).get("display_value", None) if content_info and content_info.get("publication_date") else None
        
        # Technical Info - safely get
        technical_info = {}
        try:
            technical_info = item.get("item_info", {}).get("technical_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        technical_info_str = "Available" if technical_info else None
        
        print(f"{'─'*80}")
        print(f"Item #{idx}")
        print(f"{'─'*80}")
        print(f"ASIN:         {asin}")
        print(f"Title:        {title}")
        print(f"Price:        {price}")
        if savings != "N/A":
            print(f"Savings:      {savings}")
        print(f"Reviews:      {review_count} ({star_rating} stars)")
        print(f"Sales Rank:   {sales_rank}")
        if product_group:
            print(f"Product Group: {product_group}")
        if binding:
            print(f"Binding:      {binding}")
        if color:
            print(f"Color:        {color}")
        if size:
            print(f"Size:         {size}")
        if unit_count:
            print(f"Unit Count:   {unit_count}")
        if pages_count:
            print(f"Pages:        {pages_count}")
        if publication_date:
            print(f"Pub Date:     {publication_date}")
        if technical_info_str:
            print(f"Technical Info: {technical_info_str}")
        if features:
            print(f"Features:     {len(features)} bullet points")
            for i, feature in enumerate(features[:3], 1):
                feature_text = feature[:70] + "..." if len(feature) > 70 else feature
                print(f"  {i}. {feature_text}")
            if len(features) > 3:
                print(f"  ... and {len(features) - 3} more")
        print(f"Image URL:    {image_url[:80]}..." if len(image_url) > 80 else f"Image URL:    {image_url}")
        print(f"Detail URL:   {detail_url[:80]}..." if len(detail_url) > 80 else f"Detail URL:   {detail_url}")
        print()
    
    print(f"{'='*80}\n")

print("✅ Configuration and helper functions are defined.")



✅ Configuration and helper functions are defined.


In [3]:
# Test the search API with filters
searched_items = search_api(
    browse_node_id="473357031", 
    item_page=1,
    min_price=1.0,  # Minimum price in euros
    max_price=50.0,  # Maximum price in euros
    delivery_flags=["FulfilledByAmazon"],  # Only items fulfilled by Amazon
    min_reviews_rating=4,  # Minimum 4 stars
    min_saving_percent=1  # At least 10% discount
)

# Print the results nicely
print_search_results(searched_items)


📊 Search Results Summary
Total Results: 306
Search Index: Unknown
Browse Node ID: Unknown
Browse Node Name: Unknown

Found 10 items on this page:

────────────────────────────────────────────────────────────────────────────────
Item #1
────────────────────────────────────────────────────────────────────────────────
ASIN:         B01I0IGFMK
Title:        Caricabatterie USB Amazon PowerFast da 9W per e-reader Kindle, tablet Fire - compatibile con la maggior parte dei dispositivi (inclusi smartphone, tablet, e-reader e altri)
Price:        19,99 €
Reviews:      N/A (N/A stars)
Sales Rank:   15
Browse Node:  Adattatori
Image URL:    https://m.media-amazon.com/images/I/310GUGcNyYL._SL500_.jpg
Detail URL:   https://www.amazon.it/dp/B01I0IGFMK?tag=cucinaconamor-21&linkCode=osi&th=1&psc=1

────────────────────────────────────────────────────────────────────────────────
Item #2
────────────────────────────────────────────────────────────────────────────────
ASIN:         B0DSG1NYJJ
Title:     

In [4]:
# Example: Get enriched details for ASINs from search results
if 'searched_items' in locals() and searched_items:
    asins = [item.get('asin') for item in searched_items.get('search_result', {}).get('items', []) if item.get('asin')]
    if asins:
        enriched = get_items_details(asins)
        print_enriched_items(enriched)



--- Enriching 10 items with GetItems API ---
Fetching details for batch 1/1...

📦 Enriched Items Details
Total Items: 10

────────────────────────────────────────────────────────────────────────────────
Item #1
────────────────────────────────────────────────────────────────────────────────
ASIN:         B01I0IGFMK
Title:        Caricabatterie USB Amazon PowerFast da 9W per e-reader Kindle, tablet Fire - compatibile con la maggior parte dei dispositivi (inclusi smartphone, tablet, e-reader e altri)
Price:        19,99 €
Reviews:      N/A (N/A stars)
Sales Rank:   15
Product Group: Accessori digitali
Binding:      Elettronica
Features:     4 bullet points
  1. Caribatterie originale Amazon PowerFast da 9W - 1.8A compatibile con l...
  2. Carica completamente il tuo Kindle in meno di 3 ore, Kindle Paperwhite...
  3. Ideale per l'utilizzo con il cavo USB - Micro USB incluso nei disposit...
  ... and 1 more
Image URL:    https://m.media-amazon.com/images/I/310GUGcNyYL._SL500_.jpg
Detail U